# Basic Usage Examples

This notebook demonstrates the fundamental operations of the Neon CRM SDK, providing a comprehensive introduction to the most commonly used features.

## What You'll Learn

- How to initialize and configure the Neon CRM client
- Basic account listing and searching
- Working with custom fields
- Retrieving donations
- Error handling best practices

## Prerequisites

Before running this notebook, make sure you have:
1. Installed the Neon CRM SDK: `pip install neon-crm`
2. Set up your environment variables:
   - `NEON_ORG_ID`: Your organization ID
   - `NEON_API_KEY`: Your API key
3. Or update the client initialization below with your credentials

## Import Required Libraries

In [ ]:
import os

from neon_crm import NeonClient, UserType
from neon_crm.exceptions import NeonAPIError

## Initialize the Neon CRM Client

The first step is to create a client instance. The SDK will automatically load credentials from environment variables if they're set.

In [ ]:
def initialize_client():
    """Initialize the Neon CRM client with proper error handling."""
    try:
        client = NeonClient(
            org_id=os.getenv("NEON_ORG_ID"),
            api_key=os.getenv("NEON_API_KEY"),
        )
        print("✅ Neon CRM client initialized successfully!")
        return client

    except ValueError as e:
        print(f"❌ Configuration error: {e}")
        print("Please set NEON_ORG_ID and NEON_API_KEY environment variables")
        return None


# Initialize the client
client = initialize_client()

if not client:
    print("\n⚠️  To run the examples, you need to set up your credentials:")
    print("   1. Set environment variables NEON_ORG_ID and NEON_API_KEY")
    print("   2. Or modify the client initialization code above")
else:
    print("\n🚀 Ready to run Neon CRM examples!")

## Example 1: List Accounts

This is the most fundamental operation - listing accounts from your CRM. The SDK handles pagination automatically.

In [ ]:
def list_accounts_example():
    """Demonstrate basic account listing with pagination."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("📋 Listing Accounts Example")
    print("=" * 30)

    try:
        account_count = 0

        # List individual accounts with small page size for demo
        for account in client.accounts.list(
            page_size=10, user_type=UserType.INDIVIDUAL
        ):
            account_count += 1
            first_name = account.get("firstName", "")
            last_name = account.get("lastName", "")
            email = account.get("email", "No email")

            print(f"  {account_count}. {first_name} {last_name} ({email})")

            # Limit to first 5 for demo purposes
            if account_count >= 5:
                print("  ... (showing first 5 accounts)")
                break

        if account_count == 0:
            print("  No individual accounts found")
        else:
            print(f"\n✅ Successfully listed {account_count} accounts")

    except NeonAPIError as e:
        print(f"❌ API Error: {e.message}")
        print(f"   Status Code: {e.status_code}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


# Run the example
list_accounts_example()

## Example 2: Search for Accounts

The search functionality allows you to find specific accounts using various criteria and get exactly the fields you need.

In [ ]:
def search_accounts_example():
    """Demonstrate account searching with specific criteria."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("\n🔍 Searching Accounts Example")
    print("=" * 35)

    # Search for individual accounts
    search_request = {
        "searchFields": [
            {"field": "Account Type", "operator": "EQUAL", "value": "Individual"}
        ],
        "outputFields": ["Account ID", "First Name", "Last Name", "Email"],
        "pagination": {"currentPage": 0, "pageSize": 5},  # Small page for demo
    }

    try:
        search_count = 0
        results = client.accounts.search(search_request)

        for result in results:
            search_count += 1
            account_id = result.get("Account ID", "Unknown")
            first_name = result.get("First Name", "")
            last_name = result.get("Last Name", "")
            email = result.get("Email", "No email")

            print(
                f"  {search_count}. ID {account_id}: {first_name} {last_name} ({email})"
            )

            if search_count >= 3:
                print("  ... (showing first 3 search results)")
                break

        if search_count == 0:
            print("  No accounts found matching search criteria")
        else:
            print(f"\n✅ Found {search_count} accounts matching search criteria")

    except NeonAPIError as e:
        print(f"❌ API Error: {e.message}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


# Run the example
search_accounts_example()

## Example 3: Get Available Search Fields

Before building complex searches, it's helpful to know what fields are available for searching.

In [ ]:
def search_fields_example():
    """Show available search fields for accounts."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("\n📝 Available Search Fields")
    print("=" * 30)

    try:
        search_fields = client.accounts.get_search_fields()

        print("First 10 available search fields:")
        for i, field in enumerate(search_fields[:10], 1):
            display_name = field.get("displayName", field.get("name", "Unknown"))
            field_type = field.get("dataType", "Unknown type")
            print(f"  {i:2}. {display_name} ({field_type})")

        total_fields = len(search_fields)
        if total_fields > 10:
            print(f"  ... and {total_fields - 10} more fields")

        print(f"\n✅ Total search fields available: {total_fields}")

    except NeonAPIError as e:
        print(f"❌ API Error: {e.message}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


# Run the example
search_fields_example()

## Example 4: Working with Custom Fields

Custom fields are a powerful feature that allows organizations to store additional data specific to their needs.

In [ ]:
def custom_fields_example():
    """Demonstrate working with custom fields."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("\n🏷️ Custom Fields Example")
    print("=" * 30)

    try:
        # List available custom fields
        custom_fields = list(client.accounts.list_custom_fields())

        if not custom_fields:
            print("  No custom fields found for accounts")
            return

        print(f"Found {len(custom_fields)} custom fields. First few:")

        for field in custom_fields[:3]:
            field_id = field.get("id")
            field_name = field.get("name", "Unknown")
            field_type = field.get("fieldType", "Unknown")
            print(f"  • ID: {field_id:3} | Name: '{field_name}' | Type: {field_type}")

        # Example search using the first custom field
        if custom_fields:
            first_field = custom_fields[0]
            field_id = first_field["id"]
            field_name = first_field["name"]

            print(f"\n🔍 Searching accounts with non-blank '{field_name}' values...")

            custom_search = {
                "searchFields": [{"field": field_id, "operator": "NOT_BLANK"}],
                "outputFields": ["Account ID", "First Name", "Last Name", field_id],
                "pagination": {"currentPage": 0, "pageSize": 3},
            }

            results = list(client.accounts.search(custom_search))

            if results:
                print(f"  Found {len(results)} accounts with data in this field:")
                for account in results:
                    account_id = account.get("Account ID")
                    first_name = account.get("First Name", "")
                    last_name = account.get("Last Name", "")
                    custom_value = account.get(field_name, "N/A")
                    print(
                        f"    Account {account_id}: {first_name} {last_name} = '{custom_value}'"
                    )
            else:
                print("  No accounts found with non-blank values in this field")

        print("\n✅ Custom fields example completed")

    except NeonAPIError as e:
        print(f"❌ API Error: {e.message}")
    except Exception as e:
        print(f"❌ Error in custom fields example: {e}")


# Run the example
custom_fields_example()

## Example 5: List Donations

Donations are a core part of most CRM systems. This example shows how to retrieve donation data.

In [ ]:
def donations_example():
    """Demonstrate listing donations."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("\n💰 Donations Example")
    print("=" * 25)

    try:
        donation_count = 0
        total_amount = 0.0

        # List recent donations
        for donation in client.donations.list(page_size=5):
            donation_count += 1
            amount = float(donation.get("amount", 0))
            date = donation.get("date", "Unknown date")
            donor_name = donation.get("donorName", "Anonymous")

            total_amount += amount

            print(f"  {donation_count}. ${amount:.2f} on {date} from {donor_name}")

            # Limit for demo
            if donation_count >= 3:
                print("  ... (showing first 3 donations)")
                break

        if donation_count == 0:
            print("  No donations found")
        else:
            print("\n📊 Summary:")
            print(f"   Donations shown: {donation_count}")
            print(f"   Total amount: ${total_amount:.2f}")
            print(f"   Average: ${total_amount / donation_count:.2f}")

        print("\n✅ Donations example completed")

    except NeonAPIError as e:
        print(f"❌ API Error: {e.message}")
    except Exception as e:
        print(f"❌ Unexpected error: {e}")


# Run the example
donations_example()

## Example 6: Error Handling Best Practices

Proper error handling is crucial when working with APIs. This example demonstrates comprehensive error handling.

In [ ]:
def error_handling_example():
    """Demonstrate comprehensive error handling."""
    if not client:
        print("❌ Client not initialized. Please check your credentials.")
        return

    print("\n⚠️  Error Handling Example")
    print("=" * 30)

    # Example 1: Handling invalid search criteria
    print("1. Testing invalid search criteria:")
    try:
        invalid_search = {
            "searchFields": [
                {"field": "NonExistentField", "operator": "EQUAL", "value": "test"}
            ],
            "outputFields": ["Account ID"],
        }

        list(client.accounts.search(invalid_search))
        print("   ✅ Search completed (field might exist)")

    except NeonAPIError as e:
        print(f"   ❌ API Error caught: {e.message}")
        print(f"      Status Code: {e.status_code}")
        if e.response_data:
            print(f"      Additional info: {e.response_data}")
    except Exception as e:
        print(f"   ❌ Unexpected error: {e}")

    # Example 2: Handling network/connection issues
    print("\n2. Error handling best practices:")
    print("   ✅ Always wrap API calls in try-except blocks")
    print("   ✅ Handle NeonAPIError specifically for API-related issues")
    print("   ✅ Log error details for debugging")
    print("   ✅ Provide meaningful error messages to users")
    print("   ✅ Implement retry logic for transient errors")

    print("\n✅ Error handling examples completed")


# Run the example
error_handling_example()

## Run All Basic Examples

Execute all the basic usage examples in sequence:

In [ ]:
def run_all_basic_examples():
    """Run all basic usage examples in sequence."""
    print("🚀 Running All Basic Usage Examples")
    print("=" * 45)

    if not client:
        print("❌ Cannot run examples - client not initialized")
        print("   Please check your NEON_ORG_ID and NEON_API_KEY environment variables")
        return

    examples = [
        ("List Accounts", list_accounts_example),
        ("Search Accounts", search_accounts_example),
        ("Search Fields", search_fields_example),
        ("Custom Fields", custom_fields_example),
        ("Donations", donations_example),
        ("Error Handling", error_handling_example),
    ]

    for i, (name, func) in enumerate(examples, 1):
        print(f"\n{i}. {name}")
        print("-" * 30)
        try:
            func()
            print(f"✅ {name} example completed")
        except Exception as e:
            print(f"❌ {name} example failed: {e}")

    print("\n🎉 All basic examples completed!")
    print("\n📚 Next Steps:")
    print("   • Try the Account Creation examples")
    print("   • Explore Address Management")
    print("   • Learn about Pagination for large datasets")
    print("   • Dive into Custom Fields examples")


# Uncomment to run all examples
# run_all_basic_examples()

## Cleanup

Always remember to properly close the client connection when you're done:

In [ ]:
# Clean up - close the client connection
if client:
    client.close()
    print("✅ Client connection closed successfully")
else:
    print("ℹ️  No client connection to close")

## Summary

Congratulations! You've completed the basic usage examples for the Neon CRM SDK. Here's what you learned:

### Core Concepts
1. **Client Initialization**: How to set up and configure the SDK
2. **Account Operations**: Listing and searching for accounts
3. **Search Functionality**: Using advanced search with custom criteria
4. **Custom Fields**: Working with organization-specific data
5. **Donations**: Retrieving financial data
6. **Error Handling**: Best practices for robust applications

### Key Features
- **Automatic Pagination**: The SDK handles large datasets seamlessly
- **Type Safety**: Built-in types help prevent errors
- **Flexible Search**: Powerful query capabilities
- **Custom Fields Support**: Extend the CRM with your own data

### Best Practices
- Always use environment variables for credentials
- Implement proper error handling with try-catch blocks
- Close client connections when done
- Use pagination for large datasets

## Next Steps

Now that you understand the basics, explore these advanced topics:

- **Account Creation**: Learn to create new accounts with comprehensive data
- **Address Management**: Handle multiple addresses per account
- **Custom Fields Deep Dive**: Advanced custom field operations
- **Pagination Strategies**: Efficient handling of large datasets

Happy coding with the Neon CRM SDK! 🎉